# 2. multi-agent-competitive 对抗型 - 两个大模型相互对话

首先将 wde 目录加入 python path

In [1]:
import os
import sys
from pathlib import Path
pwd = Path(os.getcwd())
sys.path.append(str(pwd.parent.parent.parent))
os.chdir(str(pwd.parent.parent.parent))

导入 RolePlayingAgent、Session 后续会介绍

In [2]:
from wde.agents import RolePlayingAgent
from wde.agents import Session

设定 llm 参数

In [3]:
llm_config = {"model": "Qwen/Qwen2-7B-Instruct", "type": "zeroclient"}

加入第一个llm驱动的角色：主持人，一位辩论赛主持人。

In [4]:
host = RolePlayingAgent(
    name="主持人",
    system_message=
    "你的名字是主持人，你是一位辩论赛主持人。"
    "最先发言，然后依次让正反方轮流发言"
    "只负责串场，不输出观点，不总结观点。",
    llm_config=llm_config,
)

加入第二个llm驱动的角色：正方辩手

In [5]:
pro = RolePlayingAgent(
    name="正方",
    system_message=
    "你是一名辩手。"
    "先于反方发言。"
    "第一轮先陈述自己立场。"
    "除第一轮外每次对话反驳反方观点。",
    llm_config=llm_config,
)

加入第三个llm驱动的角色：反方辩手

In [6]:
con = RolePlayingAgent(
    name="反方",
    system_message=
    "你是一名辩手。"
    "第一轮陈述自己立场后反拨正方观点。"
    "除第一轮外每次对话反驳正方观点。",
    llm_config=llm_config,
)

将主持人、正方辩手、反方辩手拉进同一个会话

In [7]:
session = Session(participants=[host,pro,con])

主持人先开始对话。（必须人工输入辩论题目开启辩论。）

In [8]:
session.append((host, "我是辩论赛主持人，今天辩论的题目是是否以成败论英雄，正方观点是以成败论英雄，反方观点是不以成败论英雄，请正方辩手先开始。"))

进行两轮对话

In [9]:
session.chat(max_turns=2)

主持人 (round 0)

我是辩论赛主持人，今天辩论的题目是是否以成败论英雄，正方观点是以成败论英雄，反方观点是不以成败论英雄，请正方辩手先开始。

正方 (round 0)

非常感谢您的安排，现在我很高兴代表正方进行开始陈述。我们正方的观点是“以成败论英雄”。

首先，我们要明白“英雄”的定义通常涉及勇敢、正义与领导力。在历史、文学乃至现代社会，英雄往往被赋予了超越常人的特质。然而，每个人的实践与生存环境带来众多元素，如机遇、策略、团队合作等，往往影响了结果。因此，评判英雄时仅以成功与否来决论未免过于简化的道德标准或历史评价方法。

成功不仅仅依赖于个人的品质，也受制于外部环境、机遇以及是否具备恰当的资源与帮助。拿历史上的伟大领袖来说，他们之所以被尊为英雄，很大一部分原因是由于他们在错误的决策与失败的时刻能够保持坚韧，从己失败中汲取经验，成为伟大的人。比如，拿破仑尽管最终在滑铁卢战役中失败，但他的一系列军事胜利对近代欧洲格局的形成产生了深远影响。

再者，英雄的道德与品质是长久考验的结果，而非单一事件的结果。以Carolyn Ford Johnson 为例，她在越南战争期间作为护士勇敢救助了无数士兵，即使在自己失去双腿的情况下仍然坚持救助行动。失败或危机不是衡量品质的唯一标准。在这些时刻，市民、士兵、领导人表现出的勇气、智慧与关爱，同样应被视为英雄行为。

综上所述，我们不否认成功的重要性和成就的伟大影响，但更强调的是英雄的行为、品质和他们的道德考量长度。因此，我们坚持认为“以成败论英雄”是过于片面的判断标准，不应该作为唯一的评价准则。真正的英雄精神是多维度的，包涵个人贡献的广度与深度、在困难中展现出的勇气与智慧、以及个人的价值追求和道德责任。

反方 (round 0)

首先，非常感谢辩论赛的组织者将这个主题带给我们。作为反方，我们主张的观点为“不以成败论英雄”。我们强调的是英雄的精神品质，而非仅以他们是否取得成功来评判。

“英雄”这一概念深入到人类文化中，不仅仅是因为他们在战场上的胜利，也包括在各种社会冲突、道德挑战中英勇无畏的表现。二战期间的Alleyne Johnson，他在面对纳粹的反犹太行为时，不顾个人安危去保护人民的权益，尽管他的行动并未最终带来大的正面结果改变集体历史进程，他的行为却也彰显出了伟大的勇气与坚定的道德力量。

细化这个概念，

由SummaryAgent总结对话

In [10]:
from wde.agents import SummaryAgent

In [11]:
summary_agent = SummaryAgent(
    system_message="总结下面对话",
    llm_config=llm_config)

print("总结:")
print(summary_agent.summary(session))

总结:
辩论的主题为“是否以成败论英雄”，正方与反方的辩手分别阐述了各自立场的核心观点，并围绕着英雄的品质、成功与失败间的关系以及英雄对社会的价值进行了深入讨论。

**正方的观点**：
1. **成就与品质并重**：提到的焦点在于，英雄不仅需要在取得成就时展现出卓越的品质，更在面对失败时能坚持不屈的精神，拥有智慧、勇气与道德责任，才能成为真正的英雄。
2. **历史人物案例**：通过分析拿破仑的失败与流放后的毅力、鲍勃·摩根在葡萄酒行业的贡献，以及尼尔·阿姆斯特朗对人类探月事业的象征意义，强调了不论成败，正面的精神追求与社会贡献同样被评价为英雄的条件。
3. **对社会的影响持续性**：举例马丁·路德·金的影响，说明英雄的影响力往往超越其生前至整个社会的历史和文化的改变。

**反方的观点**：
1. **品质与精神超越成败**：明确指出英雄的定义不应仅限于成败，更应看重其在社会上所展现出的道德品质、个人意志和社会贡献。
2. **认可尼尔·阿姆斯特朗、拿破仑等案例的复杂性**：在强调品质的同时，不忽视大无畏精神时间的考验，进一步说明了英雄的复杂性并不仅仅局限于最终成功与否。
3. **聚焦长期影响**：通过对鲍勃·摩根在工业领域的例证、马丁·路德·金在民权运动中的努力，强调英雄对社会的深远影响，特别是内在道德的勇气与他对社会变革的持续性贡献。

**总结与核心辩论点**：
- 双方都认可英雄概念的复杂性和多元性，正方强调成就与品质并重，反方更着眼于英雄的精神品质与对社会的积极贡献。
- 论点围绕以成败评判的价值在于英雄背后所展现的精神与对社会的长远影响，而非单一的成功作为评判标准。
- 总体上，即使双方立场存在分歧，但都重视鼓励个体勇往直前的精神和社会积极影响的价值，展现了对英雄精神的共通认可与尊重。

辩论的焦点最终落到如何全面而深入地理解英雄的概念，这对每个人如何定义与评价自己、他人乃至社会中的英雄，提供了更丰富的视角和思考。
